In [50]:
import os
import spacy
import pickle

In [51]:
filenames = [f for f in os.listdir("../data/large_data/articles_spacydocs") if ".pickle" in f]
filenames[:10]

['spacydoc_pop_id_28.pickle',
 'spacydoc_pop_id_199.pickle',
 'spacydoc_pop_id_253.pickle',
 'spacydoc_pop_id_502.pickle',
 'spacydoc_pop_id_55.pickle',
 'spacydoc_pop_id_353.pickle',
 'spacydoc_pop_id_402.pickle',
 'spacydoc_pop_id_36.pickle',
 'spacydoc_pop_id_230.pickle',
 'spacydoc_pop_id_187.pickle']

In [55]:
sourcepath = "../data/large_data/articles_spacydocs/"

In [56]:
filepath = sourcepath + "spacydoc_pop_id_17.pickle"

In [57]:
with open(filepath, "rb") as f:
    doc = pickle.load(f)

In [58]:
doc

Various scholars have argued and demonstrated that Western European populist parties have something in common. Although these parties adhere to various ideologies and employ different organizational forms and political styles, they all endorse a similar set of ideas concerning the relationship between the people and the elite. Yet despite our increasing knowledge about these parties, so far we know only very little about populist voters. Do the voter bases of populist parties also have something in common? To answer that question, I focus on the electorates of 15 prototypical populist parties from 11 Western European countries. I show that, in contrast with widely held beliefs, the electorates of populist parties do not always consist of individuals who are more likely to be ‘losers of globalization’ with Eurosceptic attitudes, low levels of political trust, and preferences for (more) direct democracy. This suggests that ‘the’ populist voter does not exist.  Keywords: populism; voting 

In [60]:
sent = [sent for sent in doc.sents][0]
sent

Various scholars have argued and demonstrated that Western European populist parties have something in common.

In [61]:
sent.ents

[Western European]

In [62]:
for ent in sent.ents:
    if ent.label_ == "PERSON":
        print(ent[-1].lemma_)

In [63]:
#ent = [ent for ent in sent.ents][0]

In [64]:
# postags = ["PROPN", "NOUN", "VERB", ]
def lemmata(spacyobject, filtered=True):
    if filtered==True:
        return  [t.lemma_ for t in spacyobject if (t.is_stop==False) & (t.is_alpha==True)]
    else:
        return  [t.lemma_ for t in spacyobject]
    # return [t.lemma_.lower() if t.pos=="PROPN" else t.lemma_ for t in spacyobject if (t.is_stop==False) & (t.is_alpha==True)]

In [65]:
lemmata(sent)

['scholar',
 'argue',
 'demonstrate',
 'western',
 'european',
 'populist',
 'party',
 'common']

In [66]:
[ent for ent in sent.ents]

[Western European]

In [67]:
ents_inds = []
entstart_dict = {}
for ent in sent.ents:
    ents_inds.extend(range(ent.start + 1, ent.end))
    entstart_dict[ent.start] = "_".join(lemmata(ent, filtered=False))

In [68]:
entstart_dict

{7: 'western_european'}

In [69]:
ents_inds

[8]

In [70]:
sent

Various scholars have argued and demonstrated that Western European populist parties have something in common.

In [71]:
def get_sentence_lemmata(sent):
    ents_inds = []
    entstart_dict = {}
    for ent in sent.ents:
        ents_inds.extend(range(ent.start + 1, ent.end))
        if ent.label_ == "PERSON":
            entstart_dict[ent.start] = ent[-1].lemma_
        else:
            entstart_dict[ent.start] = "_".join(lemmata(ent, filtered=False))
    sentence_lemmata = []
    for i in range(sent.start, sent.end):
        if i in entstart_dict.keys():
            sentence_lemmata.append(entstart_dict[i])
        else:
            if i not in ents_inds:
                t = sent[i - sent.start]
                if (t.is_stop==False) & (t.is_alpha==True):
                    sentence_lemmata.append(t.lemma_)
    return sentence_lemmata

In [72]:
sent

Various scholars have argued and demonstrated that Western European populist parties have something in common.

In [73]:
get_sentence_lemmata(sent)

['scholar',
 'argue',
 'demonstrate',
 'western_european',
 'populist',
 'party',
 'common']

In [74]:
lemmatized_sents = [" ".join(get_sentence_lemmata(sent)) for sent in doc.sents]
lemmatized_sents[:5]

['scholar argue demonstrate western_european populist party common',
 'party adhere ideology employ different organizational form political style endorse similar set idea concern relationship people elite',
 'despite increase knowledge party far know little populist voter',
 'voter basis populist party common',
 'answer question focus electorate 15 prototypical populist party 11 western_european country']

In [75]:
with open("testfile.txt", "w") as f:
    f.writelines("\n".join(lemmatized_sents))

In [76]:
!mkdir ../data/large_data/articles_lemmata

mkdir: ../data/large_data/articles_lemmata: File exists


In [77]:
filenames[0]

'spacydoc_pop_id_28.pickle'

In [78]:
sourcepath = "../data/large_data/articles_spacydocs/"
destpath = "../data/large_data/articles_lemmata/"
for filename in filenames:
    filepath = sourcepath + filename
    with open(filepath, "rb") as f:
        doc = pickle.load(f)
    lemmatized_sents = [" ".join(get_sentence_lemmata(sent)) for sent in doc.sents]
    with open(destpath + filename.replace("spacydoc", "lemmata").replace(".pickle", ".txt"), "w") as f:
        f.writelines("\n".join(lemmatized_sents))